Todo list:
- ~~book reading function~~
- morphologic analysis:
    - ~~lexical diversity calculation function~~
    - ~~parts of speech counting function (by word's initial form)~~
    - ~~parts of speech counting function (by tags, part of speech types)~~
    - ~~text dynamics calculation functiion~~
    - *top-1 for every speech part
- syntax analysis
    - reduce the amount of text
    - ~~most popular root word~~
- semantic analysis
    - stop-word cleanup
    - *automatic tag generation
- graphic demonstration
    - tag cloud

# Imports and installs

In [2]:
%pip install -q nltk pymorphy2 gensim
import nltk
import pymorphy2
import spacy
import numpy as np
import gensim.downloader

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nlp = spacy.load('ru_core_news_sm')
# word2vec_rus = gensim.downloader.load('word2vec-ruscorpora-300')

Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package punkt to /home/andrew/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/andrew/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/andrew/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Defining a few constants

In [3]:
PUNCT = ('.',',',':',';','\'','"','-','(',')','!','?','...','$','№')

# Preprocess class definition

In [11]:
class Preproc:
    def __init__(self, book_name):
        self.text = self.get_book_text(book_name)
        self.tokens = nltk.word_tokenize(self.text)
        self.morph = pymorphy2.MorphAnalyzer()
        self.tokens_no_punct = nltk.tokenize.RegexpTokenizer(r"\w+").tokenize(self.text)
        self.nlp = spacy.load('ru_core_news_sm')
        self.word2vec_rus = gensim.downloader.load('word2vec-ruscorpora-300')
        self.doc = nlp(self.text)
        self.lemm = nltk.WordNetLemmatizer()
        self.lex_diversity()
        self.make_tags()
    
    def process_stopwords(self, overwrite=False):
        stopwords = stopwords.words("russian")
        if overwrite:
            pass
        else:
            self.tokens_no_stopwords = [token for token in self.tokens if token not in stopwords]

    def get_book_text(self, name):
        with open(name, "r") as raw:
            text = ""
            for t in raw.readlines():
                text += t+"\n"
            return text
    
    def lex_diversity(self):
        unique_words_set = set(self.tokens)
        self.lex_diversity_coeff = len(unique_words_set) / len(self.tokens)

    def make_tags(self):
        self.tagged = []
        self.tokens_and_tags = []
        for token in self.tokens_no_punct:
            tag = self.morph.parse(token)[0].tag.POS
            self.tagged.append((token, tag))
            # self.tokens_and_tags.append(self.lemm.lemmatize(token).lower()+'_'+tag)
            # spacy find all собственные имена
        return self.tagged

    def text_dynamics(total_tags: list):
        verbs = total_tags.count("VERB")
        return verbs / len(total_tags)

    def text_dynamics_dict(self):
        self.count_speech_parts()
        verbs = self.tags_count["VERB"]
        total = 0
        for i in self.tags_count.keys():
            total += self.tags_count[i]
        return verbs/total
    
    def count_speech_parts(self):
        self.make_tags()
        res = dict()
        for token in self.tagged:
            if (token[1] in res.keys()) :
                res[token[1]] += 1
            else:
                res[token[1]] = 1
        self.tags_count = res

    def count_unique_words(self):
        self.make_tags()
        res = dict()
        for token in self.tagged:
            if not (token[1] in ("CONJ", "PREP", "PRCL")):
                if token[0] in res.keys():
                    res[token[0]] += 1
                else:
                    res[token[0]] = 1
        self.word_count = res
        self.word_count_sorted = dict(sorted(self.word_count.items(), key=lambda item: item[1]))
    
    @staticmethod
    def compare_books(self, book1, book2):
        text1 = self.get_book_text(book1)
        text2 = self.get_book_text(book2)

        doc1 = self.nlp(text1)
        doc2 = self.nlp(text2)
        return (doc1.similarity(doc2), np.dot(doc1.vector, doc2.vector) / (np.linalg.norm(doc1.vector) * (np.linalg.norm(doc2.vector))))

    def count_root_speech_parts(self):
        res = dict()
        for token in self.doc:
            if token.pos_ not in ("SPACE", "PUNCT"):
                if token.pos_ not in res.keys():
                    res[token.pos_] = 1
                else:
                    res[token.pos_] += 1
        self.root_count = res
        self.root_count_sorted = dict(sorted(self.root_count.items(), key=lambda item: item[1]))
    
    def fetch_tokens(self, amount=100, index=0):
        return [self.tokens[token] for token in range(index, amount)]

    def check_vectors(self, first_n_elements=100, logging=False):
        res = []
        i = 0
        while len(res) < first_n_elements:
            try:
                res.append(self.word2vec_rus[self.doc[i].lemma_+'_'+self.tagged[i][1]])
            except:
                if logging:
                    print(f"error parsing {self.doc[i].lemma_+'_'+self.tagged[i][1]}")
            i += 1
        return res
    
    def metamorph(self, first_n_elements=100):
        res = ""
        for i in self.check_vectors(first_n_elements):
            res += self.word2vec_rus.most_similar([i])[-1][0].split("_")[0] + " "
        return res

### Now onto the fun part
# Putting everything together

In [12]:
list_of_books = ["Похождения Чичикова (1921)","Белая гвардия (1922)","Роковые яйца (1924)","Собачье сердце (1925)","Мастер и маргарита (1929)","Театральный роман (1936)"]
for name in list_of_books:
    preproc = Preproc(name)
    with open(name+"_out", "w") as file:
        preproc.count_unique_words()
        preproc.count_speech_parts()
        preproc.count_root_speech_parts()
        # if name == list_of_books[0]:
        print(f"Metamorphed \"{name}\":\n",preproc.metamorph(), "\n")
        
        file.write(f"Lexical diversity: {str(preproc.lex_diversity_coeff)}\n")

        file.write("Top-3 words: \n")
        file.write(f"\t{list(preproc.word_count_sorted.keys())[-1]}\n")
        file.write(f"\t{list(preproc.word_count_sorted.keys())[-2]}\n")
        file.write(f"\t{list(preproc.word_count_sorted.keys())[-3]}\n")

        file.write(f"Text dynamics: {preproc.text_dynamics_dict()}\n")

        file.write(f"Top root speech part: {list(preproc.root_count_sorted.keys())[-1]}\n")

Metamorphed "Похождения Чичикова (1921)":
 семен терапиано скандалезный филострато чичикова шашка нарочный усы четырехаршинный бранех баба-яга скорбь jesus::christ полуявь полутень выход сверкать свечка ангел тигр топорник двигаться заездом торгашев злиться сынок полуопущенный::ресница чичикова писемский интрапсихический отворять мало-помалу билборд гнусность обязателство состоять дмитриевич дядюшка бистро выбленка обнимать император почтительно обязателство хлористый::барий четырехаршинный дмитриевич концентрический rambler::ru зачитываться гувернантка адат попятиться невыдача высокочиновный хоронюта ква::ква возмит выплачиваться таковия свара понадобиться фальшивый::монетчик облокотиться трюх-трюх старое::стариться гугб::нквд чичикова барин спассеровать сын настенный::бра возвратися финит::ля перевозить студебеккер кипрский::урегулирование просрочивать::кредиторский браслет татьянушка меч английский финит::ля четырехаршинный алмаз промемория токмачев стерва чичикова наверное товарооб

In [ ]:
preproc = Preproc(list_of_books[0])
preproc.tokens

['Михаил',
 'Булгаков',
 '.',
 'Похождения',
 'Чичикова',
 'Поэма',
 'в',
 'двух',
 'пунктах',
 'с',
 'прологом',
 'и',
 'эпилогом',
 '-',
 'Держи',
 ',',
 'держи',
 ',',
 'дурак',
 '!',
 '-',
 'Кричал',
 'Чичиков',
 'Селифану',
 '.',
 '-',
 'Вот',
 'я',
 'тебя',
 'палашом',
 '!',
 '-',
 'Кричал',
 'скакавший',
 'навстречу',
 'фельдъегерь',
 ',',
 'с',
 'усами',
 'в',
 'аршин',
 '.',
 'Не',
 'видишь',
 ',',
 'леший',
 'дери',
 'твою',
 'душу',
 ',',
 'казенный',
 'экипаж',
 '.',
 'I.',
 'Пролог',
 'Диковинный',
 'сон',
 '...',
 'Будто',
 'бы',
 'в',
 'царстве',
 'теней',
 ',',
 'над',
 'входом',
 'в',
 'которое',
 'мерцает',
 'неугасимая',
 'лампада',
 'с',
 'надписью',
 '``',
 'Мертвые',
 'души',
 "''",
 ',',
 'шутник-сатана',
 'открыл',
 'двери',
 '.',
 'Зашевелилось',
 'мертвое',
 'царство',
 'и',
 'потянулась',
 'из',
 'него',
 'бесконечная',
 'вереница',
 '.',
 'Манилов',
 'в',
 'шубе',
 'на',
 'больших',
 'медведях',
 ',',
 'Ноздрев',
 'в',
 'чужом',
 'экипаже',
 ',',
 'держиморд

In [ ]:
preproc.word2vec_rus["поэма"]

KeyError: "Key 'поэма' not present"